In [ ]:
from social_agents.agent_builder import SocialAgentBuilder
from social_agents.utils import eval_experiment
import pandas as pd
from glob import glob
import os
all_results_path = SocialAgentBuilder.ROOT_FOLDER + "experiment_results.csv"
path_ = SocialAgentBuilder.ROOT_FOLDER + "output_*.json"
time_log_path = SocialAgentBuilder.ROOT_FOLDER + "time_log.csv"

out_files = [x for x in glob(path_) if x.find("_eval_") == -1]
evaluated_files = [x for x in glob(path_) if x.find("_eval_") > -1]

data_split = "validation"
threshold = 0.6
metric = "similarity"


all_results = []
for out_ in out_files:
    eval_name = out_.replace("json", f"_eval_{metric}_{str(threshold).replace('.', '')}.json")
    if eval_name in evaluated_files:
        print("already evaluated")
        continue
    
    exp_name = os.path.basename(out_).replace("output_", "").replace(".json", "")
    eval_dict = {"experiment_name": exp_name}
    eval_dict = eval_dict | eval_experiment(submission_path=out_, data_split=data_split, threshold=threshold)

    time_log_df = pd.read_csv(time_log_path)
    if exp_name not in time_log_df.columns:
        print(f"time not logged for {exp_name}")
    else:
        eval_dict["time_mean"] = time_log_df[exp_name].mean()
        eval_dict["time_std"] = time_log_df[exp_name].std()

    try:
        all_experiments_results_df = pd.read_csv(all_results_path)
        all_experiments_results_df = all_experiments_results_df.drop(columns=[col for col in all_experiments_results_df.columns if col.startswith('Unnamed')])
    except FileNotFoundError:
        all_experiments_results_df = pd.DataFrame()
    

    all_experiments_results_df = pd.concat([all_experiments_results_df, pd.DataFrame([eval_dict])], ignore_index=True)
    all_experiments_results_df.to_csv(all_results_path, index=False)
    all_results.append(eval_dict)
    break

new_results_df = pd.DataFrame(all_results)
all_experiments_results_df = pd.read_csv(all_results_path)
summary_df = all_experiments_results_df[["experiment_name", "Useful_ratio", "3/3_ratio", "overall_punctuation", "time_mean", "time_std"]]
summary_df.to_csv(all_results_path.replace(".csv", "_summary.csv"), index=False)

Running command: python eval_scripts/evaluation.py --metric similarity --input_path data_splits/validation.json --submission_path output/elbaff_experiment/output_llama8b_social_n2_Teo_Srd.json --threshold 0.6
Distribution of the labels: Counter({'Useful': 351, 'not_able_to_evaluate': 123, 'Unhelpful': 60, 'Invalid': 24})
Distribution of the intervention punctuation: Counter({0.6666666666666666: 74, 1.0: 55, 0.3333333333333333: 38, 0: 19})
Overall punctuation 0.6290322580645163



In [ ]:
new_results_df.head()

In [ ]:
all_experiments_results_df.columns

In [ ]:
summary_df


In [ ]:
all_experiments_results_df.columns

In [ ]:
from glob import glob

files = glob("output/elbaff_experiment/final_states/*.json")

len(files)

In [ ]:
from collections import Counter
import json

fail_fs = []
numbers = []
correct_n=[]
for f in files:
    with open(f, "r") as f_:
        s = json.load(f_)
        if len(s['final_cq']['critical_questions']) != 3:
            numbers.append(len(s['final_cq']['critical_questions']))
            fail_fs.append(f)
        else:
            correct_n.append(len(s['final_cq']['critical_questions']))


Counter(numbers).values()
#Counter(correct_n)
fail_fs = list(set(fail_fs))
len(numbers)

In [ ]:
import os
for ffail in fail_fs:
    if os.path.exists(ffail):
        os.remove(ffail)
    else:
        print("Doesnt exist")

In [ ]:
deleted = fail_fs

In [ ]:
sorted(deleted)